# Embedding for the recime dataset
Oriented on: https://medium.com/spikelab/learning-embeddings-for-your-machine-learning-model-a6cb4bc6542e

Train a embedding where the model tries to predict the amount of elements over the dataset. 

In [38]:
import torch
import pandas as pd
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [13]:
class ReciMeEmbedding(torch.nn.Module):
    def __init__(self, input_dim):
        super(ReciMeEmbedding, self).__init__()

        self.emb = torch.nn.Embedding(input_dim, 300)

        self.network = torch.nn.Sequential(
            torch.nn.Linear(300, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 25),
            torch.nn.ReLU(),
            torch.nn.Linear(25, 5),
            torch.nn.ReLU(),
            torch.nn.Linear(5, 1),
            torch.nn.ReLU(),
            torch.nn.Linear(1, 1)
        )

    def forward(self, x):
        emb = self.emb(x)
        return self.network(emb)

In [2]:
import pickle
ingrDict = pickle.load(open('data/ingredients_ingredientDict.pkl', 'rb'))
ingrAmountDict = pickle.load(open('data/ingredients_ingredientAmountDict.pkl', 'rb'))

In [25]:
np.array(list(ingrAmountDict.values())).reshape(-1,1).shape

(17936, 1)

In [31]:
scaledAmounts.reshape(-1,).shape

(17936,)

In [82]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaledAmounts = scaler.fit_transform(np.array(list(ingrAmountDict.values())).reshape(-1,1))

dataList = []

for key in ingrDict.keys():
    dataList.append((int(ingrDict[key]),scaledAmounts.reshape(-1,)[list(ingrAmountDict.keys()).index(key)].astype('float32')))

In [90]:
model = ReciMeEmbedding(len(dataList)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()

In [91]:
dataSet = torch.utils.data.DataLoader(dataset=dataList)

In [92]:
def train(epoch):
    epochLoss = 0.0
    for data in dataSet:
        optimizer.zero_grad()
        data[0].to(device)
        data[1].to(device)
        y = model(data[0])
        loss = loss_fn(y[0], data[1])
        epochLoss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {epochLoss/len(dataSet)}")

In [94]:
epochs = 2

for epoch in range(1, epochs+1):
    train(epoch)

Epoch: 1, Loss: 0.00017793885318376405
Epoch: 2, Loss: 0.0001778548964262649
